# Object Oriented Programming

In this notebook I have examples to illustrate object oriented themes in Python. This notebook covers the following things:

1. Shallow and Deep copying
2. Namespaces

For each of these, I've documented what Python is doing

## Shallow and Deep Copies

We create a `colors` class which contains attributes `red`, `green` and `blue`.
We assign identifier `warmtones` to an instantiation of the class.

In [1]:
import copy

class colors():
    def __init__(self, red, green, blue):
        self._red, self._green, self._blue = red, green, blue
    def __repr__(self):
        return '(' + str(self._red) + ',' + str(self._green) + ',' + str(self._blue) + ')'

warmtones = [colors(249, 124, 43), colors(169, 163, 52)]

We add the alias `palette`. Changing the underyling object of one changes the other, as expected.

In [3]:
palette = warmtones # alias

print('before: palette: {}'.format(palette))
print('before: warmtones: {}'.format(warmtones))

palette.append(colors(0, 0, 0))

print('after: palette: {}'.format(palette))
print('after: warmtones: {}'.format(warmtones)) # both change!

before: palette: [(249,124,43), (169,163,52), (0,0,0)]
before: warmtones: [(249,124,43), (169,163,52), (0,0,0)]
after: palette: [(249,124,43), (169,163,52), (0,0,0), (0,0,0)]
after: warmtones: [(249,124,43), (169,163,52), (0,0,0), (0,0,0)]


For shallow copies, the lists are seperate, so appending to one doesn't append to the other.
The objects in the list are aliases though, so changing one _does_ change the other!

In [4]:
warmtones = [colors(249, 124, 43), colors(169, 163, 52)]
        
palette = list(warmtones) # shallow copy

print('before: palette: {}'.format(palette))
print('before: warmtones: {}'.format(warmtones))

palette.append(colors(0, 0, 0))

print('after adding element: palette: {}'.format(palette))
print('after adding element: warmtones: {}'.format(warmtones)) # now only one changes!

palette[0]._red = 255 # change attribute of entry in one list

print('after mutating element: palette: {}'.format(palette))
print('after mutating element: warmtones: {}'.format(warmtones)) # both changes!


before: palette: [(249,124,43), (169,163,52)]
before: warmtones: [(249,124,43), (169,163,52)]
after adding element: palette: [(249,124,43), (169,163,52), (0,0,0)]
after adding element: warmtones: [(249,124,43), (169,163,52)]
after mutating element: palette: [(255,124,43), (169,163,52), (0,0,0)]
after mutating element: warmtones: [(255,124,43), (169,163,52)]


With deep copies this isn't the case. We can mutate the elements of one list without mutating the other.

In [ ]:
warmtones = [colors(249, 124, 43), colors(169, 163, 52)]
palette = copy.deepcopy(warmtones) # deep copy

palette[0]._red = 255 # change attribute of entry in one list

print('palette: {}'.format(palette))
print('warmtones: {}'.format(warmtones)) # only one changes!


# Namespaces

The namespaces of classes (as well as functions) can be accessed using `dir` and `var`

In [5]:
class CreditCard():
    __slots__ = '_balance', '_name' 
    
    def __init__(self, name):
        self._balance = 0
        self._name = name
    def charge(self, amount):
        self._balance += amount

class PredatoryCreditCard(CreditCard):
    __slots__ = '_apr'
    
    OVERLIMIT_FEE = 5
    
    def __init__(self, name, apr):
        super().__init__(name)
        self._apr = apr
    
    def charge(self, price):
        success = super().charge(price)
        if not success:
            self._balance += PredatoryCreditCard.OVERLIMIT_FEE
        return success

In [6]:
cc_instance = CreditCard('john doe')
cc_instance.charge(4)

pcc_instance = PredatoryCreditCard('john doe', 0.08)
pcc_instance.charge(4)

Notice the differences in which methods appear in the class vs subclass namespaces

In [7]:
dir(cc_instance)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_balance',
 '_name',
 'charge']

In [8]:
dir(pcc_instance)

['OVERLIMIT_FEE',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_apr',
 '_balance',
 '_name',
 'charge']

In [9]:
dir(PredatoryCreditCard)

['OVERLIMIT_FEE',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_apr',
 '_balance',
 '_name',
 'charge']

In [10]:
dir(CreditCard)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_balance',
 '_name',
 'charge']